### Basic Imports

In [1]:
import argparse
import functools
import csv
from macls.predict import MAClsPredictor
from macls.utils.utils import add_arguments, print_arguments

### hyper-parameters

In [2]:
configs = 'configs/res2net.yml' # 'configs/cam++.yml'
model_path = 'models/Res2Net_Fbank/best_model/' #导出的预测模型文件路径 可选择CAM++: 'models/CAMPPlus_Fbank/best_model/'
use_gpu = True # 是否使用gpu

### predict

In [3]:
predictor = MAClsPredictor(configs=configs,
                           model_path=model_path,
                           use_gpu=use_gpu)

[2024-01-06 12:47:03.110381 INFO   ] utils:print_arguments:19 - ----------- 配置文件参数 -----------
[2024-01-06 12:47:03.111468 INFO   ] utils:print_arguments:22 - dataset_conf:
[2024-01-06 12:47:03.112365 INFO   ] utils:print_arguments:25 - 	aug_conf:
[2024-01-06 12:47:03.113000 INFO   ] utils:print_arguments:27 - 		noise_aug_prob: 0.2
[2024-01-06 12:47:03.113639 INFO   ] utils:print_arguments:27 - 		noise_dir: /openbayes/input/input0/noise
[2024-01-06 12:47:03.114238 INFO   ] utils:print_arguments:27 - 		speed_perturb: True
[2024-01-06 12:47:03.114851 INFO   ] utils:print_arguments:27 - 		volume_aug_prob: 0.2
[2024-01-06 12:47:03.115439 INFO   ] utils:print_arguments:27 - 		volume_perturb: False
[2024-01-06 12:47:03.116034 INFO   ] utils:print_arguments:25 - 	dataLoader:
[2024-01-06 12:47:03.116622 INFO   ] utils:print_arguments:27 - 		batch_size: 64
[2024-01-06 12:47:03.117203 INFO   ] utils:print_arguments:27 - 		num_workers: 4
[2024-01-06 12:47:03.117768 INFO   ] utils:print_arguments:

In [5]:
file_path = '/openbayes/home/ResNet2/test_list.txt'
title = []

with open(file_path, 'r') as file:
    # 逐行读取文件内容
    for line in file:
        # 使用制表符 '\t' 分割每一行的元素
        elements = line.strip().split('\t')
        
        # 获取每一行的第一个元素
        first_element = elements[0]
        # 打印或处理第一个元素
        title.append(first_element)
final = []         
predict = []
            
for i in range(len(title)): 
    try:
        label, score = predictor.predict(audio_data=title[i])
            # 打印或处理预测结果
        # print(f'音频：{title[i]} 的预测结果标签为：{label}，得分：{score}')
        predict.append(label)
        final.append(title[i])
    except AssertionError as e:
        continue


### Essential Defs

In [6]:
# 数据
def cut_name(name):
    c = 0
    for i in range(len(name)):
        if name[i] == "/":
            c+=1
            if c == 7:
                return name[i+1:]
    return None


def read_csv_to_dict(file_path):
    data_dict = {}
    
    with open(file_path, 'r') as csvfile:
        reader = csv.reader(csvfile)
        header = next(reader)  # 读取并保存第一行（表头）
        
    # 读取剩余行
        rows = [row for row in reader]
        for row in rows:
            if row:  # 确保行不为空
                key = row[0]  # 每行的第一个元素作为键
                value = row[-2]  # 每行的倒数第二个元素作为值
                data_dict[key] = value
    
    return data_dict

### output

In [7]:
csv_file_path = '/openbayes/home/ResNet2/UrbanSound8K.csv'  
result_dict = read_csv_to_dict(csv_file_path)
dict_label = {"air_conditioner":0,"car_horn":1,"children_playing":2,"dog_bark":3,"drilling":4,"engine_idling":5,"gun_shot":6,"jackhammer":7,"siren":8,"street_music":9}


count = 0 
for i in range(len(predict)):

    ID = str(dict_label[predict[i]])
    name = cut_name(final[i])
    if ID == result_dict[name]:
        count += 1
   
print(count/len(predict))

0.9169590643274854
